# Test de Wilcoxon Data Drift

In [1]:
# Define all method vectors
vBlue <- c(6.435, 6.505, 6.575, 7.506, 7.586, 5.974, 7.336, 6.055, 6.875, 6.845)
vIPC <- c(7.506, 7.316, 6.715, 8.136, 7.836, 6.735, 7.055, 6.645, 6.695, 6.195)
vNinguno <- c(6.885, 7.446, 6.365, 7.356, 6.995, 6.645, 6.985, 7.406, 7.486, 6.605, 7.586, 7.105, 6.835, 6.875, 6.034, 6.705, 7.706, 7.436, 6.885, 7.316)
vSTD <- c(7.446, 5.584, 6.855, 5.864, 6.535, 5.224, 6.715, 6.415, -3.653, 5.124)
vRankSimple <- c(3.873, 6.225, 6.625, 5.284, 7.045, 7.406, 6.905, 5.414, 5.944, 5.844)
vRankCero <- c(7.045, 7.266, 7.005, 7.636, 7.255, 7.125, 8.426, 7.065, 6.415, 7.716, 6.805, 7.045, 7.416, 7.055, 4.363, 6.235, 5.934, 6.705, 7.306, 6.835)
vNucleo <- c(6.225, 6.425, 7.065, 7.115, 7.135)



---



In [7]:
library(data.table)

In [ ]:
library(data.table)

# --- hasta acá tu código igual ---

# Convert results list a data.table
summary_dt <- rbindlist(lapply(results, function(r) {
  data.table(
    comparison   = r$comparison,
    test_type    = r$test_type,
    n_used       = r$n1,
    original_n   = paste0(r$original_n1, "vs", r$original_n2),
    mean1        = r$mean1,
    mean2        = r$mean2,
    p_value      = r$pvalue,
    result_code  = r$result,
    interpretation = r$interpretation
  )
}), fill = TRUE)

# Ranking table
method_means   <- sapply(methods, mean)
method_ranking <- sort(method_means, decreasing = TRUE)
ranking_dt <- data.table(
  rank             = seq_along(method_ranking),
  method           = names(method_ranking),
  mean_performance = as.numeric(method_ranking)
)

# === Export summary table ===
output_file_summary <- "summary_table.csv"
fwrite(summary_dt, output_file_summary)
cat("\nSummary table exported to:", output_file_summary, "\n")

# === Export method ranking ===
output_file_ranking <- "method_ranking.csv"
fwrite(ranking_dt, output_file_ranking)
cat("Method ranking exported to:", output_file_ranking, "\n")

cat("\nNote: Results with code 0 = no significant difference")
cat("\n      Results with code 1 = first method significantly better")
cat("\n      Results with code 2 = second method significantly better\n")


In [8]:
# Comprehensive Wilcoxon Test Comparison for All Methods
# Data Mining 2025 - Model Comparison Analysis

# USO TODOS LOS VALORES, SI n!=m bajo la dimension de n para matchear

# Create list of all methods
methods <- list(
  "Blue" = vBlue,
  "IPC" = vIPC,
  "Ninguno" = vNinguno,
  "RankSimple" = vRankSimple,
  "RankCero" = vRankCero,
  "Nucleo" = vNucleo,
  "estandarizada" = vSTD
)

# Function to perform Wilcoxon test between two vectors
perform_wilcoxon_test <- function(vec1, vec2, name1, name2) {
  # Always use paired test by truncating to minimum length
  min_length <- min(length(vec1), length(vec2))
  
  # Truncate both vectors to the same length (first n values with matching seeds)
  vec1_truncated <- vec1[1:min_length]
  vec2_truncated <- vec2[1:min_length]
  
  # Use paired test with truncated vectors
  wt <- wilcox.test(vec1_truncated, vec2_truncated, paired = TRUE)
  test_type <- "paired"
  
  pvalue <- wt$p.value
  mean1 <- mean(vec1_truncated)
  mean2 <- mean(vec2_truncated)
  
  # Determine result
  result <- 0
  interpretation <- "No significant difference"
  
  if (pvalue < 0.05) {
    if (mean1 > mean2) {
      result <- 1
      interpretation <- paste(name1, "significantly better than", name2)
    } else {
      result <- 2
      interpretation <- paste(name2, "significantly better than", name1)
    }
  }
  
  return(list(
    comparison = paste(name1, "vs", name2),
    test_type = test_type,
    n1 = length(vec1_truncated),
    n2 = length(vec2_truncated),
    original_n1 = length(vec1),
    original_n2 = length(vec2),
    mean1 = round(mean1, 3),
    mean2 = round(mean2, 3),
    pvalue = round(pvalue, 4),
    result = result,
    interpretation = interpretation
  ))
}

# Perform all pairwise comparisons
results <- list()
method_names <- names(methods)
comparison_count <- 0

cat("=== WILCOXON TEST RESULTS FOR ALL METHOD PAIRS ===\n\n")

for (i in 1:(length(methods) - 1)) {
  for (j in (i + 1):length(methods)) {
    comparison_count <- comparison_count + 1
    
    name1 <- method_names[i]
    name2 <- method_names[j]
    vec1 <- methods[[i]]
    vec2 <- methods[[j]]
    
    # Perform test
    test_result <- perform_wilcoxon_test(vec1, vec2, name1, name2)
    results[[comparison_count]] <- test_result
    
    # Print individual result
    cat("Comparison", comparison_count, ":", test_result$comparison, "\n")
    cat("  Test type:", test_result$test_type, "\n")
    cat("  Sample sizes used: n =", test_result$n1, "(truncated from", test_result$original_n1, "vs", test_result$original_n2, ")\n")
    cat("  Means:", name1, "=", test_result$mean1, ",", name2, "=", test_result$mean2, "\n")
    cat("  P-value:", test_result$pvalue, "\n")
    cat("  Result:", test_result$interpretation, "\n")
    cat("  ---\n")
  }
}

# Create summary table
cat("\n=== SUMMARY TABLE ===\n")
cat(sprintf("%-20s %-10s %-8s %-12s %-8s %-8s %-10s %s\n", 
            "Comparison", "Test", "n_used", "Original_n", "Mean1", "Mean2", "P-value", "Result"))
cat(paste(rep("-", 95), collapse=""), "\n")

for (result in results) {
  original_sizes <- paste0(result$original_n1, "vs", result$original_n2)
  cat(sprintf("%-20s %-10s %-8d %-12s %-8.3f %-8.3f %-10.4f %d\n",
              result$comparison,
              result$test_type,
              result$n1,
              original_sizes,
              result$mean1,
              result$mean2,
              result$pvalue,
              result$result))
}

# Summary statistics
cat("\n=== OVERALL SUMMARY ===\n")
cat("Total comparisons performed:", comparison_count, "\n")

significant_results <- sum(sapply(results, function(x) x$result != 0))
cat("Significant differences found:", significant_results, "\n")

cat("All tests performed as paired tests using truncated vectors\n")

# Method performance ranking by mean
method_means <- sapply(methods, mean)
method_ranking <- sort(method_means, decreasing = TRUE)

cat("\n=== METHOD RANKING BY MEAN PERFORMANCE ===\n")
for (i in 1:length(method_ranking)) {
  cat(i, ".", names(method_ranking)[i], ":", round(method_ranking[i], 3), "\n")
}

# === Export summary table ===
output_file_summary <- paste0("summary_table_", ".csv")
fwrite(summary_dt, output_file_summary)
cat("\nSummary table exported to:", output_file_summary, "\n")

# === Export method ranking ===
output_file_ranking <- paste0("method_ranking_", ".csv")
fwrite(ranking_dt, output_file_ranking)
cat("Method ranking exported to:", output_file_ranking, "\n")

cat("\nNote: Results with code 0 = no significant difference")
cat("\n      Results with code 1 = first method significantly better")
cat("\n      Results with code 2 = second method significantly better\n")


=== WILCOXON TEST RESULTS FOR ALL METHOD PAIRS ===

Comparison 1 : Blue vs IPC 
  Test type: paired 
  Sample sizes used: n = 10 (truncated from 10 vs 10 )
  Means: Blue = 6.769 , IPC = 7.083 
  P-value: 0.1602 
  Result: No significant difference 
  ---
Comparison 2 : Blue vs Ninguno 
  Test type: paired 
  Sample sizes used: n = 10 (truncated from 10 vs 20 )
  Means: Blue = 6.769 , Ninguno = 7.017 
  P-value: 0.2754 
  Result: No significant difference 
  ---
Comparison 3 : Blue vs RankSimple 
  Test type: paired 
  Sample sizes used: n = 10 (truncated from 10 vs 10 )
  Means: Blue = 6.769 , RankSimple = 6.056 
  P-value: 0.0645 
  Result: No significant difference 
  ---
Comparison 4 : Blue vs RankCero 
  Test type: paired 
  Sample sizes used: n = 10 (truncated from 10 vs 20 )
  Means: Blue = 6.769 , RankCero = 7.295 
  P-value: 0.0273 
  Result: RankCero significantly better than Blue 
  ---
Comparison 5 : Blue vs Nucleo 
  Test type: paired 
  Sample sizes used: n = 5 (truncated 

Warning message in wilcox.test.default(vec1_truncated, vec2_truncated, paired = TRUE):
“cannot compute exact p-value with ties”


Comparison 12 : Ninguno vs RankSimple 
  Test type: paired 
  Sample sizes used: n = 10 (truncated from 20 vs 10 )
  Means: Ninguno = 7.017 , RankSimple = 6.056 
  P-value: 0.0592 
  Result: No significant difference 
  ---


Warning message in wilcox.test.default(vec1_truncated, vec2_truncated, paired = TRUE):
“cannot compute exact p-value with ties”


Comparison 13 : Ninguno vs RankCero 
  Test type: paired 
  Sample sizes used: n = 20 (truncated from 20 vs 20 )
  Means: Ninguno = 7.033 , RankCero = 6.933 
  P-value: 0.7089 
  Result: No significant difference 
  ---
Comparison 14 : Ninguno vs Nucleo 
  Test type: paired 
  Sample sizes used: n = 5 (truncated from 20 vs 5 )
  Means: Ninguno = 7.009 , Nucleo = 6.793 
  P-value: 0.625 
  Result: No significant difference 
  ---
Comparison 15 : Ninguno vs estandarizada 
  Test type: paired 
  Sample sizes used: n = 10 (truncated from 20 vs 10 )
  Means: Ninguno = 7.017 , estandarizada = 5.211 
  P-value: 0.0371 
  Result: Ninguno significantly better than estandarizada 
  ---
Comparison 16 : RankSimple vs RankCero 
  Test type: paired 
  Sample sizes used: n = 10 (truncated from 10 vs 20 )
  Means: RankSimple = 6.056 , RankCero = 7.295 
  P-value: 0.0059 
  Result: RankCero significantly better than RankSimple 
  ---
Comparison 17 : RankSimple vs Nucleo 
  Test type: paired 
  Sample s

ERROR: Error: object 'summary_dt' not found


In [4]:
# Comprehensive Wilcoxon Test Comparison for All Methods
# Data Mining 2025 - Model Comparison Analysis

# USO TODOS LOS VALORES, SI n!=m hago UNPAIRED

# Create list of all methods
methods <- list(
  "Blue" = vBlue,
  "IPC" = vIPC,
  "Ninguno" = vNinguno,
  "RankSimple" = vRankSimple,
  "RankCero" = vRankCero,
  "Nucleo" = vNucleo,
  "estandarizada" = vSTD
)

# Function to perform Wilcoxon test between two vectors
perform_wilcoxon_test <- function(vec1, vec2, name1, name2) {
  # Check if vectors have same length for paired test
  if (length(vec1) != length(vec2)) {
    # Use unpaired test if lengths differ
    wt <- wilcox.test(vec1, vec2, paired = FALSE)
    test_type <- "unpaired"
  } else {
    # Use paired test if lengths are equal
    wt <- wilcox.test(vec1, vec2, paired = TRUE)
    test_type <- "paired"
  }
  
  pvalue <- wt$p.value
  mean1 <- mean(vec1)
  mean2 <- mean(vec2)
  
  # Determine result
  result <- 0
  interpretation <- "No significant difference"
  
  if (pvalue < 0.05) {
    if (mean1 > mean2) {
      result <- 1
      interpretation <- paste(name1, "significantly better than", name2)
    } else {
      result <- 2
      interpretation <- paste(name2, "significantly better than", name1)
    }
  }
  
  return(list(
    comparison = paste(name1, "vs", name2),
    test_type = test_type,
    n1 = length(vec1),
    n2 = length(vec2),
    mean1 = round(mean1, 3),
    mean2 = round(mean2, 3),
    pvalue = round(pvalue, 4),
    result = result,
    interpretation = interpretation
  ))
}

# Perform all pairwise comparisons
results <- list()
method_names <- names(methods)
comparison_count <- 0

cat("=== WILCOXON TEST RESULTS FOR ALL METHOD PAIRS ===\n\n")

for (i in 1:(length(methods) - 1)) {
  for (j in (i + 1):length(methods)) {
    comparison_count <- comparison_count + 1
    
    name1 <- method_names[i]
    name2 <- method_names[j]
    vec1 <- methods[[i]]
    vec2 <- methods[[j]]
    
    # Perform test
    test_result <- perform_wilcoxon_test(vec1, vec2, name1, name2)
    results[[comparison_count]] <- test_result
    
    # Print individual result
    cat("Comparison", comparison_count, ":", test_result$comparison, "\n")
    cat("  Test type:", test_result$test_type, "\n")
    cat("  Sample sizes: n1 =", test_result$n1, ", n2 =", test_result$n2, "\n")
    cat("  Means:", name1, "=", test_result$mean1, ",", name2, "=", test_result$mean2, "\n")
    cat("  P-value:", test_result$pvalue, "\n")
    cat("  Result:", test_result$interpretation, "\n")
    cat("  ---\n")
  }
}

# Create summary table
cat("\n=== SUMMARY TABLE ===\n")
cat(sprintf("%-20s %-10s %-8s %-8s %-8s %-8s %-10s %s\n", 
            "Comparison", "Test", "n1", "n2", "Mean1", "Mean2", "P-value", "Result"))
cat(paste(rep("-", 90), collapse=""), "\n")

for (result in results) {
  cat(sprintf("%-20s %-10s %-8d %-8d %-8.3f %-8.3f %-10.4f %d\n",
              result$comparison,
              result$test_type,
              result$n1,
              result$n2,
              result$mean1,
              result$mean2,
              result$pvalue,
              result$result))
}

# Summary statistics
cat("\n=== OVERALL SUMMARY ===\n")
cat("Total comparisons performed:", comparison_count, "\n")

significant_results <- sum(sapply(results, function(x) x$result != 0))
cat("Significant differences found:", significant_results, "\n")

paired_tests <- sum(sapply(results, function(x) x$test_type == "paired"))
unpaired_tests <- sum(sapply(results, function(x) x$test_type == "unpaired"))
cat("Paired tests:", paired_tests, "\n")
cat("Unpaired tests:", unpaired_tests, "\n")

# Method performance ranking by mean
method_means <- sapply(methods, mean)
method_ranking <- sort(method_means, decreasing = TRUE)

cat("\n=== METHOD RANKING BY MEAN PERFORMANCE ===\n")
for (i in 1:length(method_ranking)) {
  cat(i, ".", names(method_ranking)[i], ":", round(method_ranking[i], 3), "\n")
}

cat("\nNote: Results with code 0 = no significant difference")
cat("\n      Results with code 1 = first method significantly better")
cat("\n      Results with code 2 = second method significantly better\n")

=== WILCOXON TEST RESULTS FOR ALL METHOD PAIRS ===

Comparison 1 : Blue vs IPC 
  Test type: paired 
  Sample sizes: n1 = 10 , n2 = 10 
  Means: Blue = 6.769 , IPC = 7.083 
  P-value: 0.1602 
  Result: No significant difference 
  ---


Warning message in wilcox.test.default(vec1, vec2, paired = FALSE):
“cannot compute exact p-value with ties”


Comparison 2 : Blue vs Ninguno 
  Test type: unpaired 
  Sample sizes: n1 = 10 , n2 = 20 
  Means: Blue = 6.769 , Ninguno = 7.033 
  P-value: 0.1795 
  Result: No significant difference 
  ---
Comparison 3 : Blue vs RankSimple 
  Test type: paired 
  Sample sizes: n1 = 10 , n2 = 10 
  Means: Blue = 6.769 , RankSimple = 6.056 
  P-value: 0.0645 
  Result: No significant difference 
  ---


Warning message in wilcox.test.default(vec1, vec2, paired = FALSE):
“cannot compute exact p-value with ties”


Comparison 4 : Blue vs RankCero 
  Test type: unpaired 
  Sample sizes: n1 = 10 , n2 = 20 
  Means: Blue = 6.769 , RankCero = 6.933 
  P-value: 0.3909 
  Result: No significant difference 
  ---
Comparison 5 : Blue vs Nucleo 
  Test type: unpaired 
  Sample sizes: n1 = 10 , n2 = 5 
  Means: Blue = 6.769 , Nucleo = 6.793 
  P-value: 1 
  Result: No significant difference 
  ---
Comparison 6 : Blue vs estandarizada 
  Test type: paired 
  Sample sizes: n1 = 10 , n2 = 10 
  Means: Blue = 6.769 , estandarizada = 5.211 
  P-value: 0.0645 
  Result: No significant difference 
  ---


...

Comparison 17 : RankSimple vs Nucleo 
  Test type: unpaired 
  Sample sizes: n1 = 10 , n2 = 5 
  Means: RankSimple = 6.056 , Nucleo = 6.793 
  P-value: 0.111 
  Result: No significant difference 
  ---
Comparison 18 : RankSimple vs estandarizada 
  Test type: paired 
  Sample sizes: n1 = 10 , n2 = 10 
  Means: RankSimple = 6.056 , estandarizada = 5.211 
  P-value: 0.625 
  Result: No significant difference 
  ---


Warning message in wilcox.test.default(vec1, vec2, paired = FALSE):
“cannot compute exact p-value with ties”


Comparison 19 : RankCero vs Nucleo 
  Test type: unpaired 
  Sample sizes: n1 = 20 , n2 = 5 
  Means: RankCero = 6.933 , Nucleo = 6.793 
  P-value: 0.6343 
  Result: No significant difference 
  ---


Warning message in wilcox.test.default(vec1, vec2, paired = FALSE):
“cannot compute exact p-value with ties”


Comparison 20 : RankCero vs estandarizada 
  Test type: unpaired 
  Sample sizes: n1 = 20 , n2 = 10 
  Means: RankCero = 6.933 , estandarizada = 5.211 
  P-value: 0.0094 
  Result: RankCero significantly better than estandarizada 
  ---
Comparison 21 : Nucleo vs estandarizada 
  Test type: unpaired 
  Sample sizes: n1 = 5 , n2 = 10 
  Means: Nucleo = 6.793 , estandarizada = 5.211 
  P-value: 0.1292 
  Result: No significant difference 
  ---

=== SUMMARY TABLE ===
Comparison           Test       n1       n2       Mean1    Mean2    P-value    Result
------------------------------------------------------------------------------------------ 
Blue vs IPC          paired     10       10       6.769    7.083    0.1602     0
Blue vs Ninguno      unpaired   10       20       6.769    7.033    0.1795     0
Blue vs RankSimple   paired     10       10       6.769    6.056    0.0645     0
Blue vs RankCero     unpaired   10       20       6.769    6.933    0.3909     0
Blue vs Nucleo       unpaired